# Content-based recommendations (using cosine similarity)
This notebook shows the process used to calculate recommendations for user based on their previous ratings and in features extracted from content texts available in +TV4E platform.

Library requirements:
* Pandas
* NumPy
* Scikit-learn 
* NLTK (w/ portuguese stop words)

In [125]:
# for Python 2: use print only as a function
from __future__ import print_function

### Part 1 - Examine the data

In [126]:
# read CSV using a relative path
import pandas as pd
path = 'ratings.csv'
df_ratings = pd.read_csv(path)
path = 'videos.csv'
df_videos = pd.read_csv(path)

In [127]:
# show dataframe first row
df_ratings.head()

,user_id,content_id,rating,rating_timestamp,rating_type
0,1,1096,0.23,2017-08-23 16:01:01+00:00,explicit
1,1,1099,0.88,2017-08-23 16:01:01+00:00,explicit


In [128]:
#show the shape of dataframe
df_ratings.shape

(2, 5)

In [129]:
# show overall statistics of the dataframe
df_ratings.describe()

,user_id,content_id,rating
count,2.0,2.00000,2.000000
mean,1.0,1097.50000,0.555000
std,0.0,2.12132,0.459619
min,1.0,1096.00000,0.230000
25%,1.0,1096.75000,0.392500
50%,1.0,1097.50000,0.555000
75%,1.0,1098.25000,0.717500
max,1.0,1099.00000,0.880000


In [130]:
# filter ratings from user 1 and show statistics
df_ratings_user = df_ratings[(df_ratings.user_id==1)]
df_ratings_user.describe()

,user_id,content_id,rating
count,2.0,2.00000,2.000000
mean,1.0,1097.50000,0.555000
std,0.0,2.12132,0.459619
min,1.0,1096.00000,0.230000
25%,1.0,1096.75000,0.392500
50%,1.0,1097.50000,0.555000
75%,1.0,1098.25000,0.717500
max,1.0,1099.00000,0.880000


### Part 2 - Clean ratings data

In [131]:
# rtemoving unused columns
df_ratings.drop('rating_type', axis=1, inplace=True)
df_ratings.drop('rating_timestamp', axis=1, inplace=True)
df_ratings

,user_id,content_id,rating
0,1,1096,0.23
1,1,1099,0.88


In [132]:
ratings_matrix = df_ratings.pivot_table(index=['user_id'], columns=['content_id'], values=['rating'])
ratings_matrix                                     

rating      
content_id   1096  1099
user_id                
1            0.23  0.88

### Part 3 - Vectorize texts

In [133]:
# import and retrieve portuguese stop words
# stop words are not considered as token as usually they carry no meaning (!)
from nltk.corpus import stopwords
portuguese_stop_words = stopwords.words('portuguese')

In [134]:
# import and instantiate CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2, max_df=0.5, stop_words=portuguese_stop_words)

In [135]:
# fit (train) the vectorizer with the corpus from video text contents
tfidf_matrix = tfidf.fit_transform(df_videos['text_contents'])
tfidf_matrix.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [136]:
tfidf.get_feature_names()

['00',
 '00 horas',
 '00 junto',
 '000',
 '000 euros',
 '000 vacinas',
 '04',
 '05',
 '06',
 '09',
 '10',
 '10 acesso',
 '10 anos',
 '10 associações',
 '10 cuidados',
 '10 dias',
 '10 edição',
 '10 envolvidos',
 '10 julho',
 '10 mil',
 '10 países',
 '10 viaturas',
 '100',
 '100 mil',
 '100 milhões',
 '100 pode',
 '101',
 '101 exército',
 '1011',
 '1011 utentes',
 '109',
 '10h00',
 '10h30',
 '11',
 '11 00',
 '11 concelhos',
 '11 julho',
 '11 meses',
 '11 viaturas',
 '112',
 '117',
 '117 inspetores',
 '11h30',
 '12',
 '12 500',
 '12 anos',
 '12 factos',
 '12 horas',
 '12 julho',
 '12 meses',
 '12 mil',
 '120',
 '120 dias',
 '13',
 '13 anos',
 '13 julho',
 '13 maio',
 '130',
 '134',
 '14',
 '14 16',
 '14 arguidos',
 '14 dias',
 '14 estruturas',
 '14 horas',
 '14 julho',
 '1400',
 '1400 elementos',
 '142',
 '144',
 '15',
 '15 anos',
 '15 dias',
 '15 horas',
 '15 julho',
 '15 mil',
 '15 países',
 '15 setembro',
 '150',
 '150 horas',
 '1500',
 '1539',
 '1539 situações',
 '1550',
 '156',
 '15

### Part 4 - Build up user profile

In [137]:
# create dict content_id ==>> tfidf weights
tfidf_tokens = {}
tfidf_array = tfidf_matrix.toarray()
line_count = 0
for idx, row in df_videos.iterrows():
    tfidf_tokens[row.id] = tfidf_array[line_count]
    line_count += 1
tfidf_tokens

{788: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 789: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 791: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 793: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 794: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 801: array([ 0.        ,  0.        ,  0.        , ...,  0.04117254,
         0.        ,  0.        ]),
 802: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 818: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 820: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 821: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 822: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 836: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 837: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 851: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 853: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 855: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 856: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 857: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 858: array([ 0.,  0.,  0., ...,  0.,  0.,  

In [138]:
# define user ratings
user_ratings = df_ratings[(df_ratings.user_id==1)]
user_ratings

,user_id,content_id,rating
0,1,1096,0.23
1,1,1099,0.88


In [139]:
# created weighted user profile vector (dotproduct of vectors of items consumed and user ratings)
#   "In the original implementation, the profile was the sum of the item-tag vectors of all items 
#   the user has rated positively (>= 3.5 stars). This approach was later improved with weighted 
#   user profile (with the older implementation commented out for reference). Weighted profile is 
#   computed with weighted sum of the item vectors for all items, with weights being based on the 
#   user's rating."
#   See: http://eugenelin89.github.io/recommender_content_based/
user_profile = [0] * len(tfidf.get_feature_names())
for i in range(len(user_profile)):
    for idx, row in user_ratings.iterrows():
        user_profile[i] += row.rating * tfidf_tokens[row.content_id][i]
    #user_profile = [v/len(user_ratings) for v in user_profile] # weight-ing user vector (?)
# normalize user profile vector
import numpy as np
user_profile = user_profile / np.linalg.norm(user_profile)
user_profile

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

### Part 5 - Calculate similarities between user profile and contents

In [140]:
# apply cosine similarity between user profile vector and content vectors
# See: http://eugenelin89.github.io/recommender_content_based/
from math import sqrt
def cosine_similarity(x, y):
    def square_rooted(v):
        return round(sqrt(sum([a * a for a in v])), 3)
   
    numerator = sum(a * b for a, b in zip(x, y))
    denominator = square_rooted(x) * square_rooted(y)
    return numerator/float(denominator)

estimated_user_ratings = {}
for content_id, token_weights in tfidf_tokens.items(): 
    if content_id not in user_ratings.content_id.values: # not calculating for contents already consumed 
        estimated_user_ratings[content_id] = cosine_similarity(user_profile, token_weights)
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-5:-1]

[(1101, 0.28655363294874181),
 (904, 0.26634325420091498),
 (1093, 0.24469900160682989),
 (1256, 0.24070078175034049)]

In [141]:
# Using the dotproduct of user profile vector, content IDF and content vector
# See: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/
estimated_user_ratings = {}
for content_id, token_weights in tfidf_tokens.items():
    if content_id not in user_ratings.content_id.values: # removing contents already consumed 
        estimated_user_ratings[content_id] = 0
        for i in range(len(token_weights)):
            if token_weights[i] > 0:
                estimated_user_ratings[content_id] += user_profile[i] * tfidf.idf_[i] * token_weights[i]
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-5:-1]

[(1101, 1.0466900976226285),
 (904, 0.89202187214982498),
 (1256, 0.87141748114351958),
 (1093, 0.86129594787974439)]